In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd

# Load model and encoders
model = joblib.load(r"D:\Projects\New folder\edudrop\backend\models\dropout_risk_model.pkl")
label_encoders = joblib.load(r"D:\Projects\New folder\edudrop\backend\models\label_encoders.pkl")

# Example MongoDB string data
student = {
    'gender': 'Female',
    'age': 17,
    'caste': 'General',
    'area': 'Urban',
    'standard': 10,
    'state': 'Karnataka',
    'school': 'Government',
    'maritalStatus': 'Single',
    'course': 'Science',
    'previousQualification': 'Secondary',
    'motherQualification': 'Graduate',
    'fatherQualification': 'Graduate',
    'motherOccupation': 'Labor',
    'fatherOccupation': 'Clerk',
    'specialNeeds': 'No',
    'debtor': 'No',
    'tuitionUpToDate': 'Yes',
    'scholarshipHolder': 'Yes',
    'profileCompleted': 'Yes',
    'daysSinceRegistration': 180,
    'totalActivityMinutes': 5000,
    'attendanceRate': 0.9,
    'averageScore': 95.0
}

# Create DataFrame
input_df = pd.DataFrame([student])

# Apply the saved label encoders
for col, encoder in label_encoders.items():
    if col in input_df:
        input_df[col] = input_df[col].apply(lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1)

# Predict
pred = model.predict(input_df)[0]
risk_label = label_encoders['riskLabel'].inverse_transform([pred])[0]
print("Predicted Risk:", risk_label)


Predicted Risk: medium


In [1]:
import numpy as np
import sklearn

print(np.__version__)  # Should print 1.21.5
print(sklearn.__version__)  # Should print 1.1.3


1.21.5
1.1.3


In [10]:
import pymongo
from datetime import datetime

# MongoDB setup
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["edudrop"]
collection = db["solution_pathways"]

# Paste your CSV content as a string (raw, no quoting needed)
csv_data = """
Category,Scheme/Program Name,Implementing Agency/Ministry,Target Group/Eligibility,Key Features/Benefits,Impact/Goal,Website URL
Government Scheme,Right to Education Act (RTE),Ministry of Education,"Children aged 6-14","Free and compulsory education, no capitation fees, no screening for admission","Universal access to elementary education",https://rteportal.nic.in/
Scholarship,National Means Cum Merit Scholarship,Dept. of School Education & Literacy,"Class 9–12, family income ≤ Rs. 1.5 lakh/year, merit-based","Annual scholarship for meritorious students from economically weaker sections","Encourage continuation of studies after Class 8",https://scholarships.gov.in/
Scholarship,Post-Matric Scholarship for Minorities,Ministry of Minority Affairs,"Minority students, Class 11–PhD, income ≤ Rs. 2 lakh","Financial support for higher studies","Promote higher education among minorities",https://scholarships.gov.in/
Scholarship,National Fellowship & Scholarship for Higher Education of ST Students,Ministry of Tribal Affairs,"ST students, higher education","Fellowship for M.Phil/PhD, scholarships for UG/PG","Promote higher education among ST students",https://scholarships.gov.in/
Scholarship,Merit-cum-Means Scholarship for Professional & Technical Courses,Ministry of Minority Affairs,"Minority students, technical/professional courses, income ≤ Rs. 2 lakh","Tuition and maintenance allowance","Promote professional education among minorities",https://scholarships.gov.in/
Support Program,Early Warning System (Pilot),USAID/Partner NGOs,"At-risk students in pilot schools","Data-driven identification of at-risk students, parent engagement via voice messaging","Timely intervention to prevent dropouts",https://www.usaid.gov/india/education
Support Program,In-school Enrichment Activities,USAID/Partner NGOs,"Students in pilot schools","Extracurricular and cooperative learning activities","Improve self-confidence, school engagement, reduce dropout",https://www.usaid.gov/india/education
Support Program,Inclusive Education Initiatives,Ministry of Education,"Marginalized, disabled, girls, minorities","Curriculum updates, teacher training, infrastructure improvements, targeted interventions","Sustain student interest, retention, universal access",https://dsel.education.gov.in/inclusive-education
"""

# Clean and split lines
lines = [line.strip() for line in csv_data.strip().split("\n") if line.strip()]
header = lines[0].split(",")

docs = []

for line in lines[1:]:
    parts = line.split(",")
    
    # Handle fields with commas — join the known field spans manually
    if len(parts) >= 10:
        doc = {
            "dataset": "india",
            "category": parts[0],
            "title": parts[1],
            "implementingAgency": parts[2],
            "targetGroup": parts[3],
            "description": ", ".join(parts[4:-2]),  # Merge all description parts
            "impact": parts[-2],
            "link": parts[-1],
            "addedOn": datetime.utcnow()
        }
        docs.append(doc)
    else:
        print("⚠️ Skipping malformed line:", line)

# Insert into MongoDB
if docs:
    result = collection.insert_many(docs)
    print(f"✅ Inserted {len(result.inserted_ids)} records into MongoDB.")
else:
    print("⚠️ No valid records to insert.")


⚠️ Skipping malformed line: Government Scheme,Right to Education Act (RTE),Ministry of Education,"Children aged 6-14","Free and compulsory education, no capitation fees, no screening for admission","Universal access to elementary education",https://rteportal.nic.in/
⚠️ Skipping malformed line: Scholarship,National Means Cum Merit Scholarship,Dept. of School Education & Literacy,"Class 9–12, family income ≤ Rs. 1.5 lakh/year, merit-based","Annual scholarship for meritorious students from economically weaker sections","Encourage continuation of studies after Class 8",https://scholarships.gov.in/
⚠️ Skipping malformed line: Scholarship,Post-Matric Scholarship for Minorities,Ministry of Minority Affairs,"Minority students, Class 11–PhD, income ≤ Rs. 2 lakh","Financial support for higher studies","Promote higher education among minorities",https://scholarships.gov.in/
⚠️ Skipping malformed line: Scholarship,National Fellowship & Scholarship for Higher Education of ST Students,Ministry of Tr

C:\Users\Hari\AppData\Local\Temp\ipykernel_3212\2962387392.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "addedOn": datetime.utcnow()
